In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MySparkApp") \
    .getOrCreate()

spark

In [25]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CSVtoDF").getOrCreate()
from pyspark.sql.functions import col

In [26]:
df1 = spark.read.option("header", "true").csv(r"C:\Users\nkuma309\OneDrive - Capgemini\Desktop\PopulationAnalysis\FINAL_POPULATION_RURAL.csv")
df2 = spark.read.option("header", "true").csv(r"C:\Users\nkuma309\OneDrive - Capgemini\Desktop\PopulationAnalysis\FINAL_POPULATION_URBAN.csv")

df1_alias = df1.alias("left")
df2_alias = df2.alias("right")

In [27]:
df = df1_alias.join(
    df2_alias,
    df1_alias["COUNTRY ID"] == df2_alias["COUNTRY ID"]
).drop(
    df2_alias["COUNTRY"]
).drop(
    df2_alias["COUNTRY ID"]
).drop(
    df2_alias["CONTINENT"]
).drop(
    df2_alias["DEVELOPMENT_CATEGORY"]
).drop(
    df2_alias["GOVERNANCE"]
)

In [28]:
df.show()

+--------------------+----------+---------+--------------+--------------------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
|             COUNTRY|COUNTRY ID|CONTINENT|   DEVELOPMENT|          GOVERNANCE| TYPE|D_1950|D_1955|D_1960|D_1965|D_1970|D_1975|D_1980|D_1985|D_1990|D_1995|D_2000|D_2005|D_2010|D_2015|D_2020|D_2025|D_2030|D_2035|D_2040|D_2045|D_2050| TYPE|D_1950|D_1955|D_1960|D_1965|D_1970|D_1975|D_1980|D_1985|D_1990|D_1995|D_2000|D_2005|D_2010|D_2015|D_2020|D_2025|D_2030|D_2035|D_2040|D_2045|D_2050|
+--------------------+----------+---------+--------------+--------------------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+---

In [19]:
maindf = df.fillna(0) 

In [20]:
maindf.show()

+--------------------+----------+---------+--------------+--------------------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
|             COUNTRY|COUNTRY ID|CONTINENT|   DEVELOPMENT|          GOVERNANCE| TYPE|D_1950|D_1955|D_1960|D_1965|D_1970|D_1975|D_1980|D_1985|D_1990|D_1995|D_2000|D_2005|D_2010|D_2015|D_2020|D_2025|D_2030|D_2035|D_2040|D_2045|D_2050| TYPE|D_1950|D_1955|D_1960|D_1965|D_1970|D_1975|D_1980|D_1985|D_1990|D_1995|D_2000|D_2005|D_2010|D_2015|D_2020|D_2025|D_2030|D_2035|D_2040|D_2045|D_2050|
+--------------------+----------+---------+--------------+--------------------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+---

In [30]:
population = maindf.cache()

In [31]:
population.rdd.getNumPartitions()

1

In [32]:
# Question 11: Total population rural vs urban

r = population.select(
    col("COUNTRY"),
    col("left.D_2025").alias("RURAL"),
    col("right.D_2025").alias("URBAN")
)

In [34]:
resultQ11 = r.na.drop()

resultQ11.show()

+--------------------+-----+-----+
|             COUNTRY|RURAL|URBAN|
+--------------------+-----+-----+
|             Burundi| 9118| 1674|
|             Comoros|  722|  320|
|            Djibouti|  247|  919|
|             Eritrea| 5440| 2174|
|            Ethiopia|86212|23758|
|               Kenya|41086|17969|
|          Madagascar|18246|12972|
|              Malawi|19636| 4577|
|           Mauritius|  779|  603|
|             Mayotte|  146|  159|
|          Mozambique|20754|11685|
|             R�union|   38|  935|
|              Rwanda|11929| 3854|
|          Seychelles|   37|   54|
|             Somalia| 7777| 6375|
|         South Sudan|11378| 3186|
|              Uganda|40805|11526|
|United Republic o...|46805|24074|
|              Zambia|11370| 9602|
|            Zimbabwe| 8993| 7668|
+--------------------+-----+-----+
only showing top 20 rows



In [36]:
# Question 12:	Country Wise Governance

resultQ12 = population.select("COUNTRY", "GOVERNANCE")

resultQ12.show()

+--------------------+--------------------+
|             COUNTRY|          GOVERNANCE|
+--------------------+--------------------+
|             Burundi|presidential repu...|
|             Comoros|federal president...|
|            Djibouti|semi-presidential...|
|             Eritrea|presidential repu...|
|            Ethiopia|federal parliamen...|
|               Kenya|presidential repu...|
|          Madagascar|semi-presidential...|
|              Malawi|presidential repu...|
|           Mauritius|parliamentary rep...|
|             Mayotte|presidential repu...|
|          Mozambique|presidential repu...|
|             R�union|     communist state|
|              Rwanda|presidential repu...|
|          Seychelles|presidential repu...|
|             Somalia|federal parliamen...|
|         South Sudan|presidential repu...|
|              Uganda|presidential repu...|
|United Republic o...|presidential repu...|
|              Zambia|presidential repu...|
|            Zimbabwe|semi-presi

In [37]:
# Question 13:  RURAL population of Countries which are More Developed

resultQ13 = population.select(
    col("COUNTRY"),
    col("left.D_2025").alias("RURAL")
).filter(
    col("DEVELOPMENT") == "MORE DEVELOPED"
).na.fill("0")

resultQ13.show()

+-------------------+-----+
|            COUNTRY|RURAL|
+-------------------+-----+
|              Japan| 4581|
|            Belarus| 1827|
|           Bulgaria| 1379|
|     Czech Republic| 2797|
|            Hungary| 2415|
|             Poland|14645|
|Republic of Moldova| 1404|
|            Romania| 9379|
| Russian Federation|32712|
|           Slovakia| 2455|
|            Ukraine|11691|
|    Channel Islands|  104|
|            Denmark|  658|
|            Estonia|  378|
|      Faroe Islands|   29|
|            Finland|  809|
|            Iceland|   19|
|            Ireland| 1717|
|        Isle of Man|   42|
|             Latvia|  660|
+-------------------+-----+
only showing top 20 rows

